In [ ]:
import os
import joblib
from os.path import join

import numpy as np

In [ ]:
root_folder = "../results_eval_tune"

In [ ]:
model_dict = {}

files = os.listdir(root_folder)
files_other = ['Dec-17-20:59:35_xgb_5k_freeze_10_5_clinical',
 'Dec-11-13:58:40_xgb_5k_freeze_10_5_blood',
 'Dec-11-13:59:07_xgb_5k_freeze_10_5_imaging_pca',
  'Dec-18-20:16:19_xgb_5k_freeze_10_5_clinical_blood',
  'Dec-18-23:13:58_xgb_5k_freeze_10_5_clinical_imaging_pca',
 'Dec-17-23:54:04_log_5k_freeze_10_5_clinical',
 'Dec-12-14:32:32_log_0k_freeze_10_5_blood',
 'Dec-12-14:34:11_log_0k_freeze_10_5_imaging_pca',
  'Dec-19-02:16:22_log_5k_freeze_10_5_clinical_blood',
  'Dec-19-05:13:04_log_5k_freeze_10_5_clinical_imaging_pca',
  #'Dec-18-16:23:31_mlp_0k_freeze_10_5_clinical',
  'Dec-19-15:59:52_mlp_1k_freeze_10_5_clinical',
        
]

files_other = ['Dec-22-14:21:47_xgb_5k_freeze_10_5_clinical_blood']

files_other = ['Jan-06-00:35:58_xgb_5k_freeze_10_5_clinical']

files_other = ['Jan-07-22:47:10_xgb_5k_freeze_10_5_clinical',
         'Jan-08-03:05:05_xgb_5k_freeze_10_5_blood',
         'Jan-08-19:10:28_xgb_5k_freeze_10_5_sparse_img',
         
         'Jan-08-13:02:26_log_5k_freeze_10_5_clinical',
         'Jan-08-17:36:36_log_5k_freeze_10_5_blood',
         'Jan-09-07:29:04_log_5k_freeze_10_5_sparse_img',
         
         'Jan-09-13:32:30_mlp_1k_freeze_10_5_clinical',
         
        ]

for file in files:
    if "xgb" not in file:
        continue
    #if "clinical" not in file:
    #    continue
    #if "clinical" in file and "blood" in file:
    #    continue
    #if "imaging_pca" in file:
    #    continue
    
    #if "freeze" not in file:
    #    continue
    eval_tuning_path = join(root_folder, file)
    study_list =  os.listdir(eval_tuning_path)
    study_list = [study for study in study_list if study.endswith(".pkl") and study != "cfg.pkl"]
    #if len(study_list) != 10:
    #    continue
    print(file, len(study_list))

    names = ["hyperparams", "eval_score", "score_dict", "y_pred_logits", "y_pred_binary", "y_true"]
    models = [joblib.load(join(eval_tuning_path, study))[-1] for study in study_list]
    model_dict[file] = models
    # print hyperparams:
    if "opt" in file:
        for study in study_list:
            hypers = joblib.load(join(eval_tuning_path, study))[0]
            print(hypers["df"])


In [ ]:
feat_imp_dict = {}
for key in model_dict:
    if isinstance(model_dict[key][0], np.ndarray):
        continue
    print(key)
    feat_imp_dict[key] = [model.feature_importances_ for model in model_dict[key]]

In [ ]:
for key in feat_imp_dict:
    print(key)
    mean_feats = np.mean(feat_imp_dict[key], axis=0)
    std_feats = np.std(feat_imp_dict[key], axis=0)

    print(mean_feats.sum())

In [ ]:
for key in feat_imp_dict:
    num_feats_key = feat_imp_dict[key][0].shape[0]
    print(key)
    print(num_feats_key)
    print()
    assert num_feats_key > 4, f'{key}, {num_feats_key}'

In [ ]:
#feat_imp_dict["Dec-12-14:24:23_xgb_5k_freeze_10_5_clinical_blood"][0].shape

In [ ]:
#feat_imp_dict["Dec-12-18:33:22_xgb_5k_freeze_10_5_clinical_imaging_pca"][0].shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
all_dts = ["clinical", "blood", "imaging_pca", "sparse_img"]

In [ ]:
dts = ["clinical", "sparse_img", "blood"]
#dts = ["clinical"]

In [ ]:
import sys
import importlib
sys.path.append("..")
import src.utils.load_data_utils
importlib.reload(src.utils.load_data_utils)
from src.utils.load_data_utils import get_data

df = "yeo_Y/z/median/uni_clip_0.9999/multi_clip_N"
x_train, y_train, x_eval, y_eval, n_features, feature_names, class_weights = get_data(df_name=df, split='no-split', nf=0, v=0,
             blood=("blood" in dts), static=1, clinical=("clinical" in dts), imaging=0, imaging_pca=("imaging" in dts), miss_feats=0,
             sparse_img=("sparse_img" in dts)
             )

In [ ]:
data_first = x_train[0]
data_len = len(data_first)
print("Num feats: ", data_first.shape[1])
print("All:", data_len, "Dev:", int(data_len * 0.8), "Test", int(data_len * 0.2))
print("Train split size: ", int(data_len * 0.8 * 0.9), "Val split size: ", int(data_len * 0.8 * 0.1))

In [ ]:
len(feature_names)

In [ ]:
study_name = [key for key in feat_imp_dict if all([dt in key for dt in dts]) and not any([dt in key for dt in [other_dt for other_dt in all_dts if other_dt not in dts]])]
study_name = study_name[0]
study_name

In [ ]:
feat_imp_dict[study_name][0].shape

In [ ]:
rename = True
if rename:
    for dt in all_dts:
        feature_names = [col[len(dt) + 1:] if col.startswith(dt) else col for col in feature_names]
    feature_names = [col.replace("_", " ") for col in feature_names]
    feature_names = [col.replace(" nan", " - Missingness Feature") for col in feature_names]
    feature_names = [col[0].upper() + col[1:] for col in feature_names]

    feature_names = [col[9:] if "clinical" in col else col for col in feature_names]
    feature_names = [col[6:] if "blood" in col else col for col in feature_names]

In [ ]:
df = pd.DataFrame(data=feat_imp_dict[study_name], columns=feature_names)

In [ ]:
# Exclude two blood lab values that should have been filtered out before training:
df = df[[col for col in df.columns if col not in ("Zonulin", "Site of leptin measurement (laboratory)")]]

In [ ]:
mean_df = df.mean(axis=0).sort_values(ascending=True)
# remove everything above 90% explained importance
print(len(mean_df))
mean_df = mean_df[mean_df.cumsum() > 0.1]
print(len(mean_df))
mean_df = mean_df * 100
if dts == ["sparse_img"]:
    plt.figure(figsize=(6, 2), dpi=300)
elif dts == ["clinical"]:
    plt.figure(figsize=(6, 5), dpi=300)
elif "blood" in dts:
    plt.figure(figsize=(6, 10), dpi=300)
else:
    plt.figure(figsize=(6, 5), dpi=300)

plt.xlabel("Percentage of importance")
fig = mean_df.plot.barh()
plt.tight_layout()
name = "_".join(dts)
print(name)
plt.savefig(name + "importances.jpg")
mean_df.to_csv(name + "importances.csv")

In [ ]:
mean_df = df.mean(axis=0).sort_values(ascending=True)
mean_df = mean_df[mean_df != 0]
mean_df = mean_df * 100
plt.figure(figsize=(8, 13), dpi=300)
plt.xlabel("Percentage of importance")
fig = mean_df.plot.barh()
plt.tight_layout()
#name = "_".join(dts)
#plt.savefig(name + "importances.jpg")
#mean_df.to_csv(name + "importances.csv")

In [ ]:
len(mean_df)